In [1]:
%pip install langchain langchain-text-splitters langchain-community bs4


  Using cached langchain-1.0.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached langchain_text_splitters-1.0.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
  Using cached langchain_core-1.0.4-py3-none-any.whl.metadata (3.5 kB)
  Using cached langgraph-1.0.2-py3-none-any.whl.metadata (7.4 kB)
  Using cached pydantic-2.12.4-py3-none-any.whl.metadata (89 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langsmith-0.4.41-py3-none-any.whl.metadata (14 kB)
  Using cached pyyaml-6.0.3-cp310-cp310-win_amd64.whl.metadata (2.4 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached langgraph_checkpoint-3.0.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached langgraph_prebuilt-1.0.2-py3-none-any.whl.metadata (5.0 kB)
  Using cac

In [2]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [3]:
%pip install -U "langchain[openai]"

  Using cached langchain_openai-1.0.2-py3-none-any.whl.metadata (1.8 kB)
  Using cached openai-2.7.1-py3-none-any.whl.metadata (29 kB)
  Using cached tiktoken-0.12.0-cp310-cp310-win_amd64.whl.metadata (6.9 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.11.1-cp310-cp310-win_amd64.whl.metadata (5.3 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached regex-2025.11.3-cp310-cp310-win_amd64.whl.metadata (41 kB)
Using cached langchain_openai-1.0.2-py3-none-any.whl (81 kB)
Using cached openai-2.7.1-py3-none-any.whl (1.0 MB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached jiter-0.11.1-cp310-cp310-win_amd64.whl (207 kB)
Using cached tiktoken-0.12.0-cp310-cp310-win_amd64.whl (879 kB)
Using cached regex-2025.11.3-cp310-cp310-win_amd64.whl (277 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)

   ---------------------------------------- 0/7 [tqdm]
   ---------------------------------------- 0/7 [tqdm]
   ---

In [4]:
%pip install python-dotenv

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()              
api_key = os.getenv("OPENAI_API_KEY")

from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4.1")

In [2]:
%pip install -U "langchain-openai"

Note: you may need to restart the kernel to use updated packages.


In [3]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [4]:
%pip install -U "langchain-core"

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [6]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


Total characters: 43047


In [7]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 63 sub-documents.


In [9]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['6fb99624-2f20-43fb-9cf7-59b5cdbf2c09', '364ea7c0-da40-4273-a9ba-961de61faf6a', '105eeddd-cf5d-40ad-b1e5-1b7142e1a158']


In [10]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [11]:
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model, tools, system_prompt=prompt)

In [15]:
query = (
    "What is microservices?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is microservices?

Once you get the answer, look up common extensions of that method.
================================== Ai Message ==================================

Certainly! Let’s work step by step, as per your process.

Step 1: Understanding Microservices  
Microservices is an architectural style that structures an application as a collection of small, loosely coupled, and independently deployable services. Each service in microservices architecture implements a specific business capability and communicates with other services over defined APIs, generally HTTP REST or messaging systems.

Core Characteristics:

- Each microservice is independently deployable.
- Services are typically organized around business capabilities.
- Each service can be developed, deployed, and scaled independently.
- Microservices communicate over lightweight protocols (HTTP/REST, gRPC, messaging queues).
- Technology h

In [18]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message


agent = create_agent(model, tools=[], middleware=[prompt_with_context])

In [16]:
query = "What is microservices?"
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is microservices?
================================== Ai Message ==================================

**Microservices** is an architectural style used in software development that structures an application as a collection of loosely coupled, independently deployable, and modular services. Each service is designed to perform a specific business function and communicates with other services through well-defined APIs, often using protocols like HTTP or messaging queues.

### Key Features of Microservices

1. **Single Responsibility**: Each microservice handles a specific piece of functionality, such as user authentication, payment processing, or product catalog management.
2. **Independence**: Services can be developed, deployed, updated, and scaled independently from one another.
3. **Technology Diversity**: Each service can be built with a different technology stack (programming languages, databases, fr